In [45]:
!pip install git+https://github.com/Cupcakus/pafy youtube-dl

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/Cupcakus/pafy to /state/partition1/job-18202880/pip-req-build-2_6sv7bp
  Created wheel for pafy: filename=pafy-0.5.5-py2.py3-none-any.whl size=35726 sha256=049a2db97570f82ee926ab6ffb66e42e6540c1d7598287b565be47ec9d5c7139
  Stored in directory: /state/partition1/job-18202880/pip-ephem-wheel-cache-kwp7_glg/wheels/5d/e7/a9/54c8d39d8a8ea411f44c2a402193bc2fc0d0d51ad82936c4f7
Successfully built pafy
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [46]:
import os
import pafy
import torch
import cv2
import tqdm
import matplotlib as mpl
import matplotlib.cm as cm
import numpy as np
import PIL.Image as pil
from time import sleep
from torchvision import transforms
from Models.EncoderModel import EncoderModelConvNeXt
from Models.DecoderModel import DepthDecoderModel, PoseDecoderModel

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [50]:
path = os.path.join("models-convnext-tuned-espcn", "weights_19")

In [51]:
enc = EncoderModelConvNeXt()
encoderDict = torch.load(os.path.join(path, "encoder.pth"), map_location=device)
height = encoderDict.pop("height")
width = encoderDict.pop("width")
enc.load_state_dict(encoderDict)
enc.to(device)
enc.eval()

EncoderModelConvNeXt(
  (encoder): ConvNeXt(
    (features): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
      )
      (1): Sequential(
        (0): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (1): Permute()
            (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            (3): Linear(in_features=96, out_features=384, bias=True)
            (4): GELU()
            (5): Linear(in_features=384, out_features=96, bias=True)
            (6): Permute()
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (1): Permute()
            (2): LayerNorm((96,), eps=1e-06

In [52]:
depthDecoder = DepthDecoderModel(enc.numChannels)
depthDecoder.load_state_dict(torch.load(os.path.join(path, "decoder.pth"), map_location=device))
depthDecoder.to(device)
depthDecoder.eval()

DepthDecoderModel(
  (decoder): ModuleList(
    (0): ConvBlock(
      (pad): ReflectionPad2d((1, 1, 1, 1))
      (conv): Conv2d(768, 256, kernel_size=(3, 3), stride=(1, 1))
      (act): ELU(alpha=1.0)
    )
    (1): ConvBlock(
      (pad): ReflectionPad2d((1, 1, 1, 1))
      (conv): Conv2d(640, 256, kernel_size=(3, 3), stride=(1, 1))
      (act): ELU(alpha=1.0)
    )
    (2): ConvBlock(
      (pad): ReflectionPad2d((1, 1, 1, 1))
      (conv): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1))
      (act): ELU(alpha=1.0)
    )
    (3): ConvBlock(
      (pad): ReflectionPad2d((1, 1, 1, 1))
      (conv): Conv2d(320, 128, kernel_size=(3, 3), stride=(1, 1))
      (act): ELU(alpha=1.0)
    )
    (4): ConvBlock(
      (pad): ReflectionPad2d((1, 1, 1, 1))
      (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
      (act): ELU(alpha=1.0)
    )
    (5): ConvBlock(
      (pad): ReflectionPad2d((1, 1, 1, 1))
      (conv): Conv2d(160, 64, kernel_size=(3, 3), stride=(1, 1))
      (act): E

In [53]:
video_in = "testInput2.mp4"
video_out = "output-" + video_in

In [54]:
video_in = cv2.VideoCapture(video_in)
fps = video_in.get(cv2.CAP_PROP_FPS)
width = int(video_in.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video_in.get(cv2.CAP_PROP_FRAME_HEIGHT))
length = int(video_in.get(cv2.CAP_PROP_FRAME_COUNT))
video_out = cv2.VideoWriter(video_out, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, 2*height))
ok = True
with torch.no_grad():
    for i in tqdm.tqdm(range(0, length, 1)):
        video_in.set(cv2.CAP_PROP_POS_FRAMES, i)
        while ok:
            ok, frame = video_in.read()
            if not ok:
                print("Waiting ... ")
                sleep(1)
                ok = True
            else:
                ok = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        original_height, original_width = frame.shape[:2]
        orig_img = frame
        frame = cv2.resize(frame, (640, 192), interpolation = cv2.INTER_AREA)
        frame = torch.FloatTensor(frame).permute(2, 0, 1).unsqueeze(0).cuda()
        features = enc(frame)
        outputs = depthDecoder(features)
        disp = outputs[("disp", 0)]
        disp_resized = torch.nn.functional.interpolate(
                        disp, (original_height, original_width), mode="bilinear", align_corners=False)
        disp_resized_np = disp_resized.squeeze().cpu().numpy()
        vmax = np.percentile(disp_resized_np, 95)
        normalizer = mpl.colors.Normalize(vmin=disp_resized_np.min(), vmax=vmax)
        mapper = cm.ScalarMappable(norm=normalizer, cmap='magma')
        colormapped_im = (mapper.to_rgba(disp_resized_np)[:, :, :3] * 255).astype(np.uint8)
        wm_frame = np.vstack([orig_img, colormapped_im])
        wm_frame = cv2.cvtColor(wm_frame, cv2.COLOR_BGR2RGB)
        video_out.write(wm_frame)
        ok = True
video_out.release()

100%|█████████▉| 4549/4569 [12:11<00:02,  7.39it/s]

Waiting ... 
Waiting ... 
Waiting ... 
Waiting ... 
Waiting ... 
Waiting ... 


100%|█████████▉| 4549/4569 [12:16<00:03,  6.17it/s]


KeyboardInterrupt: 

In [55]:
video_out.release()